# feature engineering template 

In [2]:
import featuretools as ft
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

/Users/billy/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
df_train_target = pd.read_csv('dataset/train_target.csv')
df_test_target = pd.read_csv('dataset/offline_test.csv')
df_online_train = pd.read_csv('dataset/online_train.csv')
df_offline_train = pd.read_csv('dataset/offline_train.csv')

In [37]:
df_target = df_target[['User_id','Merchant_id','Coupon_id','Date_received','Date','datediff','target']]
df_target['Date_received'] = pd.to_datetime(df_target['Date_received'])
df_target['Date'] = pd.to_datetime(df_target['Date'])
df_target['Month'] = df_target['Date_received'].values.astype('datetime64[M]')

In [5]:
df_offline_train['use_coupon'] = ((df_offline_train['Coupon_id'].notna()) &(df_offline_train['Date'].notna())).astype(int)
df_offline_train['purchase'] = df_offline_train['Date'].notna().astype(int)
df_offline_train['give_coupon'] =  (df_offline_train['Coupon_id'].notna()).astype(int)
df_offline_train['Date_received'] = pd.to_datetime(df_offline_train['Date_received'])
df_offline_train['Date'] = pd.to_datetime(df_offline_train['Date'])
df_offline_train['datediff'] = (df_offline_train['Date'] - df_offline_train['Date_received'] ).apply(lambda x:x.days)
df_offline_train['']

In [38]:
df_test_target

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,discount_type,uniform_discount_rate,promotion_condition,promotion_amonut
0,4129537,450,9983,30:5,1.0,2016-07-12,1,0.833333,30,5
1,6949378,1300,3429,30:5,NaN,2016-07-06,1,0.833333,30,5
2,2166529,7113,6928,200:20,5.0,2016-07-27,1,0.900000,200,20
3,2166529,7113,1808,100:10,5.0,2016-07-27,1,0.900000,100,10
4,6172162,7605,6500,30:1,2.0,2016-07-08,1,0.966667,30,1
...,...,...,...,...,...,...,...,...,...,...
113635,5828093,5717,10418,30:5,10.0,2016-07-16,1,0.833333,30,5
113636,6626813,1699,7595,30:1,NaN,2016-07-07,1,0.966667,30,1
113637,6626813,7321,7590,50:5,NaN,2016-07-12,1,0.900000,50,5
113638,4547069,760,13602,30:5,0.0,2016-07-17,1,0.833333,30,5


In [11]:
y_date_start = pd.to_datetime('2016-05-01')
y_date_end = y_date_start + pd.DateOffset(months=1)

In [13]:
df_target_tmp = df_target[(df_target['Date_received'] >= y_date_start) & (df_target['Date_received'] < y_date_end)]

In [16]:
df_target_tmp

,User_id,Merchant_id,Coupon_id,Date_received,Date,datediff,target
0,1439408,4663,11002.0,2016-05-28,NaT,NaN,0
4,1439408,2632,8591.0,2016-05-16,2016-06-13,28.0,-1
7,2029232,450,1532.0,2016-05-30,NaT,NaN,0
8,2029232,6459,12737.0,2016-05-19,NaT,NaN,0
16,253750,6901,2366.0,2016-05-18,NaT,NaN,0
...,...,...,...,...,...,...,...
1053258,2876768,6901,2107.0,2016-05-31,NaT,NaN,0
1053259,4056416,7177,3469.0,2016-05-14,NaT,NaN,0
1053277,212662,3021,3739.0,2016-05-04,2016-05-08,4.0,1
1053279,212662,3021,3739.0,2016-05-08,2016-06-02,25.0,-1


In [17]:
user = df_target_tmp[['User_id']].drop_duplicates()
merchant = df_target_tmp[['Merchant_id']].drop_duplicates()
coupon = df_target_tmp[['Coupon_id']].drop_duplicates()

In [19]:
end = y_date_start
start = end-pd.DateOffset(month=3)

In [20]:
df_offline_tmp = df_offline_train[((df_offline_train['Date'] < end) & (df_offline_train['Date'] >= start))\
                | ((df_offline_train['Date'].isna()) & (df_offline_train['Date_received'] < end) 
                   & (df_offline_train['Date_received'] >= start))] 

In [21]:
es = ft.EntitySet(id='offline_test')
es = es.entity_from_dataframe(entity_id='merchant',
                             dataframe=merchant,
                             index='Merchant_id',
                             variable_types={'Merchant_id':ft.variable_types.Id})

es = es.entity_from_dataframe(entity_id='offline',
                             dataframe=df_offline_tmp,
                             #time_index='Date',
                              index='Transaction_id',
                             make_index=True,
                             variable_types={'Merchant_id':ft.variable_types.Id,
                                            'User_id':ft.variable_types.Id,
                                            'Coupon_id':ft.variable_types.Id,
                                            'give_coupon':ft.variable_types.Categorical,
                                            'use_coupon':ft.variable_types.Categorical,
                                            'purchase':ft.variable_types.Categorical,
                                            'discount_type':ft.variable_types.Categorical})
es['offline']['give_coupon'].interesting_values=[1]
es['offline']['use_coupon'].interesting_values=[1]
es['offline']['purchase'].interesting_values=[1]
es['offline']['discount_type'].interesting_values=[0,1]
es = es.add_relationship(ft.Relationship(es['merchant']['Merchant_id'],es['offline']['Merchant_id']))

feature_matrix, features_list = ft.dfs(entityset=es,
                                        target_entity='merchant',
                                        agg_primitives=['mean','count','max','min','num_unique','mode'],
                                        trans_primitives=['weekday'],
                                       primitive_options={
                                           'mode': {'ignore_variables': {
                                               'offline': ['Coupon_id', 'User_id', 'give_coupon','use_coupon','purchase','discount_type']
                                           }
                                                   },
                                           'num_unique': {'ignore_variables': {
                                               'offline': [ 'give_coupon','use_coupon','purchase','discount_type']
                                           }
                                                   }
                                       },
                                       where_primitives=['count','mean','mode','num_unique'],
                                        #instance_ids = [4663],
                                        #include_cutoff_time=False,
                                      #cutoff_time_in_index=True
                                      )

In [22]:
feature_matrix['coupon_used_rate'] = feature_matrix['COUNT(offline WHERE use_coupon = 1)'] / feature_matrix['COUNT(offline WHERE give_coupon = 1)']
feature_matrix['purchase_coupon_used_rate'] = feature_matrix['COUNT(offline WHERE use_coupon = 1)'] / feature_matrix['COUNT(offline WHERE purchase = 1)']

In [23]:
feature_matrix[['coupon_used_rate','purchase_coupon_used_rate']].describe()

,coupon_used_rate,purchase_coupon_used_rate
count,1996.000000,3488.000000
mean,0.125616,0.046982
std,0.206571,0.128358
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.018693,0.000000
75%,0.169912,0.033427
max,1.000000,1.000000
